In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import time
import pandas as pd

ctx= ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
def getStockList(text_):
    stockList = list()
    url = "https://money.cnn.com/data/hotstocks/"
    html = urllib.request.urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, "lxml")
    header = soup.find('h3',text=text_)
    table = header.find_next_sibling('table')
    rows = table.find_all('tr')
    for tr in rows:
        if tr.find('td') != None:
            td = tr.find('td')
            ticker_symbol = td.a.text
            ticker_link = td.a.get('href')
            title = td.span.text
            stockList.append((ticker_symbol,title,"https://money.cnn.com/"+ticker_link))
    return stockList

In [3]:
def getStockInfo(stockList):
    stockInfoList = []
    for stock in stockList:
        stockDetails = []
        url = stock[2]
        html = urllib.request.urlopen(url, context=ctx).read()
        soup = BeautifulSoup(html, "lxml")
        header = soup.find('h3',text="Today’s Trading")
        table = header.find_next_sibling()
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            for col in columns:
                stockDetails.append(col.text)
        stockInfoList.append((stock[0],stock[1],stockDetails[0],stockDetails[1],stockDetails[2],stockDetails[3],stockDetails[6],stockDetails[7],stockDetails[10],stockDetails[11]))
    return stockInfoList

In [4]:
mostActiveStocks = getStockList("Most Actives")
gainerStocks = getStockList("Gainers")
losersStocks = getStockList("Losers")

In [5]:
mostActiveStocksInfo = getStockInfo(mostActiveStocks)
gainerStocksInfo = getStockInfo(gainerStocks)
losersStocksInfo = getStockInfo(losersStocks)

HTTPError: HTTP Error 503: between bytes timeout

In [ ]:
#creating a dataframe to store all the details regarding stocks
df = pd.DataFrame(columns = ['Category', 'CompanyCode', 'CompanyName','Open','PreviousClose','Volume','MarketCap'])
#iterating through each active stock details and appending to dataframe
for content in mostActiveStocksInfo:
    df = df.append({'Category':'Most Actives', 'CompanyCode':content[0], 'CompanyName':content[1], 'Open':content[5], 'PreviousClose':content[3], 'Volume':content[7], 'MarketCap':content[9]}, 
                ignore_index = True)
#iterating through each gainer stock details and appending to dataframe
for content in gainerStocksInfo:
    df = df.append({'Category':'Gainers', 'CompanyCode':content[0], 'CompanyName':content[1], 'Open':content[5], 'PreviousClose':content[3], 'Volume':content[7], 'MarketCap':content[9]}, 
                ignore_index = True)
#iterating through each loser stock details and appending to dataframe
for content in losersStocksInfo:
    df = df.append({'Category':'Losers', 'CompanyCode':content[0], 'CompanyName':content[1], 'Open':content[5], 'PreviousClose':content[3], 'Volume':content[7], 'MarketCap':content[9]}, 
                ignore_index = True)

In [ ]:
#printing the dataframe
df

In [ ]:
#getting the current time for name of file
fileName = time.strftime("%Y%m%d-%H%M%S"+".csv")
#conevrting the dataframe and save it csv file
df.to_csv(fileName, index=False)